In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg')

%matplotlib notebook

import matplotlib.pyplot as plt
from rotcoil import *

In [ ]:
# Define serial numbers, one for each magnet

serials = [
    '002', '004', '005', '006', '007', '008', '009', 
    '010', '011', '012', '013', '014', '015', '016', '017', '018', '019',
    '020', '021', '023', '024', '025', '026', '027', '028', '029',
    '030', '031', '032', '033', '034', '035', '036', '037', '038', '039',
    '040', '041', '042', '043', '044', '045', '046', '047', '048', '049',
    '050', '051', '052', '053', '054', '055', '056', '057', '058', '059',
    '060', '061', '062', '063', '064', '065', '066', '067', '068', '069',
    '070', '071', '072', '073', '074', '075', '076', '077', '078', '079',
    '080', '081',
]

# Load all data
meas = dict()
for i in range(len(serials)):
    sn = serials[i]
    meas[sn] = RotCoilMeas_SIQuadQ14(sn)
    print('index: {:02d}, serial_number: {}, data sets: {}'.format(i, sn, meas[sn].data_sets))

In [ ]:
# Magnetic Center
c, g = meas[serials[56]].get_rampup('M1')
print(c)
print(g)

In [ ]:
# Random Integrated Quadrupole Errors

tmpl_meas = meas[serials[0]]

def get_gl_set(current_index):
    c, g = [], []
    for sn in meas:
        m = meas[sn]
        ct, gt = m.get_rampup('M1')
        c.append(ct[current_index])
        g.append(gt[current_index])
    g_avg = np.mean(g)
    g_std = np.std(g)
    return g_avg, g_std, c, g


currents, _ = tmpl_meas.get_rampup('M1')
errors, cs, gs = [], [], []
for i in range(len(currents)):
    g_avg, g_std, c, g = get_gl_set(i)
    error = [100*(gv - g_avg)/g_avg for gv in g]
    fmtstr = 'current: {:+8.3f} [A], rms_error: {:7.4f} [%], max_error: {:7.4f} [%]'
    print(fmtstr.format(np.mean(c), abs(100*g_std/g_avg), max(np.abs(error))))
    errors.append(error)
    cs.append(c)
    gs.append(g)
    
# Plot Integrated Quadrupole Error for maximum current

data = errors[-1]
spec_rms = tmpl_meas.spec_main_intmpole_rms_error
avg, std = np.mean(data), np.std(data)
plt.plot(data, 'ob')
plt.plot((1,len(data)), (spec_rms, spec_rms), '--r')
plt.plot((1,len(data)), (-spec_rms, -spec_rms), '--r')
#lt.xticks(range(len(serial_numbers)), serial_numbers, rotation='vertical')
plt.title('Magnets Integrated Gradient Error at Maximum Current')
plt.xlabel('Serial Number Index')
plt.ylabel('Difference from average [%]')
plt.show()